
# Ejemplo de JOINs en SQL con SQLite
En este cuaderno, aprenderemos los diferentes tipos de **JOINs** en SQL de manera visual utilizando **SQLite**. 
Trabajaremos con dos tablas simples:
1. **Clientes**: Contiene información sobre los clientes.
2. **Pedidos**: Contiene información sobre los pedidos realizados por clientes.

---

## Configuración inicial
Instalaremos las bibliotecas necesarias y crearemos una base de datos SQLite directamente en este entorno.


In [2]:

# Importamos sqlite3 para trabajar con bases de datos SQLite
import sqlite3
import pandas as pd
#import mysql.connector

""" Conectar a la base de datos
conn = mysql.connector.connect(
    host="localhost",
    user="tu_usuario",
    password="tu_contraseña",
    database="empresa"
)"""


# Creamos una base de datos en memoria
conexion = sqlite3.connect(":memory:")
consulta = conexion.cursor()

# Función auxiliar para mostrar una tabla
def mostrar_tabla(query):
    df = pd.read_sql_query(query, conexion)
    return df


## Creación de Tablas

In [3]:

# Crear tabla Clientes
consulta.execute('''
CREATE TABLE Clientes (
    ID INTEGER PRIMARY KEY,
    Nombre TEXT
);
''')

# Insertar datos en Clientes
consulta.executemany('INSERT INTO Clientes (ID, Nombre) VALUES (?, ?)', [
    (1, 'Juan'),
    (2, 'Ana'),
    (3, 'Luis'),
    (4, 'Maria')
])

# Crear tabla Pedidos
consulta.execute('''
CREATE TABLE Pedidos (
    ID INTEGER PRIMARY KEY,
    ID_Cliente INTEGER,
    Producto TEXT
);
''')

# Insertar datos en Pedidos
consulta.executemany('INSERT INTO Pedidos (ID, ID_Cliente, Producto) VALUES (?, ?, ?)', [
    (1, 1, 'Manzanas'),
    (2, 1, 'Naranjas'),
    (3, 2, 'Plátanos'),
    (4, 5, 'Uvas')  # Cliente no existente
])

conexion.commit()

# Mostrar las tablas
print("Tabla Clientes:")
display(mostrar_tabla("SELECT * FROM Clientes"))

print("Tabla Pedidos:")
display(mostrar_tabla("SELECT * FROM Pedidos"))


Tabla Clientes:


,ID,Nombre
0,1,Juan
1,2,Ana
2,3,Luis
3,4,Maria


Tabla Pedidos:


,ID,ID_Cliente,Producto
0,1,1,Manzanas
1,2,1,Naranjas
2,3,2,Plátanos
3,4,5,Uvas


## 1. INNER JOIN o JOIN
<div style="display: flex; align-items: center;">
    <img src="./imagenes/join.png" alt="Join" style="margin-right: 10px;" />
    <div>
        <pre><code>SELECT columna1, columna2, ...
    FROM tabla1
    JOIN tabla2
    ON tabla1.columna_relacionada = tabla2.columna_relacionada;</code></pre>
    </div>
</div>
<p>En SQL, un <b>JOIN</b> se utiliza para combinar filas de dos o más tablas en función de una condición relacionada. El <code>INNER JOIN</code>, o simplemente <code>JOIN</code>, es el tipo de unión más común. Este tipo de JOIN devuelve solo las filas que tienen coincidencias en ambas tablas.</p>

In [6]:

# INNER JOIN: Muestra solo los registros con coincidencias en ambas tablas
query = '''
SELECT Clientes.ID, Clientes.Nombre, Pedidos.Producto
FROM Clientes
JOIN Pedidos ON Clientes.ID = Pedidos.ID_Cliente;
'''
print("Resultado del INNER JOIN o JOIN:")
mostrar_tabla(query)


Resultado del INNER JOIN o JOIN:


,ID,Nombre,Producto
0,1,Juan,Manzanas
1,1,Juan,Naranjas
2,2,Ana,Plátanos


## 2. LEFT JOIN
<div style="display: flex; align-items: center;">
    <img src="./imagenes/join left.png" alt="Left Join" style="margin-right: 10px;" />
    <div>
        <pre><code>SELECT columna1, columna2, ...
        FROM tabla1 # Tabla izquierda
        LEFT JOIN tabla2 
        ON tabla1.columna_relacionada = tabla2.columna_relacionada;</code></pre>
    </div>
</div>
<p>Un <b>LEFT JOIN</b> devuelve todos los registros de la tabla izquierda (en este caso <code>tabla1</code>), junto con los registros coincidentes de la tabla derecha (<code>tabla2</code>). Si no hay coincidencia, los valores de la tabla derecha se establecen en <code>NULL</code>.</p>

In [7]:

# LEFT JOIN: Muestra todos los registros de la tabla izquierda (Clientes)
# y los coincidentes de la tabla derecha (Pedidos)
query = '''
SELECT Clientes.ID, Clientes.Nombre, Pedidos.Producto
FROM Clientes
LEFT JOIN Pedidos ON Clientes.ID = Pedidos.ID_Cliente;
'''
print("Resultado del LEFT JOIN:")
mostrar_tabla(query)


Resultado del LEFT JOIN:


,ID,Nombre,Producto
0,1,Juan,Manzanas
1,1,Juan,Naranjas
2,2,Ana,Plátanos
3,3,Luis,None
4,4,Maria,None


## 3. RIGHT JOIN
<div style="display: flex; align-items: center;">
    <img src="./imagenes/join right.png" alt="Right Join" style="margin-right: 10px;" />
    <div>
        <pre><code>SELECT columna1, columna2, ...
        FROM tabla1 #Tabla derecha
        RIGHT JOIN tabla2
        ON tabla1.columna_relacionada = tabla2.columna_relacionada;</code></pre>
    </div>
</div>
<p>Un <b>RIGHT JOIN</b> devuelve todos los registros de la tabla derecha (en este caso <code>tabla1</code>), junto con los registros coincidentes de la tabla izquierda (<code>tabla2</code>). Si no hay coincidencia, los valores de la tabla izquierda se establecen en <code>NULL</code>.</p>

In [ ]:

# SQLite no soporta RIGHT JOIN nativamente, pero podemos simularlo
query = '''
SELECT Pedidos.ID, Clientes.Nombre, Pedidos.Producto
FROM Pedidos
LEFT JOIN Clientes ON Pedidos.ID_Cliente = Clientes.ID;
'''
print("Resultado simulado del RIGHT JOIN:")
mostrar_tabla(query)


## 4. FULL OUTER JOIN o FULL JOIN
<div style="display: flex; align-items: center;">
    <img src="./imagenes/full join.png" alt="Full Join" style="margin-right: 10px;" />
    <div>
        <div>
        <pre><code>SELECT columna1, columna2, ...
        FROM tabla1
        FULL OUTER JOIN tabla2
        ON tabla1.columna_relacionada = tabla2.columna_relacionada;</code></pre>
        </div>
        <div>
        <pre><code>SELECT columna1, columna2, ...
        FROM tabla1
        FULL JOIN tabla2
        ON tabla1.columna_relacionada = tabla2.columna_relacionada;</code></pre>
        </div>
    </div>
</div>
<p>
 Ambos términos se refieren a una operación de unión que devuelve todas las filas cuando hay una coincidencia en una de las tablas. Si no hay coincidencias, las columnas de la tabla que no tiene coincidencias tendrán valores NULL.
</p>

In [9]:

# FULL OUTER JOIN: Unión de LEFT JOIN y RIGHT JOIN
query = '''
SELECT Clientes.ID, Clientes.Nombre, Pedidos.Producto
FROM Clientes
LEFT JOIN Pedidos ON Clientes.ID = Pedidos.ID_Cliente
UNION
SELECT Pedidos.ID_Cliente, Clientes.Nombre, Pedidos.Producto
FROM Pedidos
LEFT JOIN Clientes ON Pedidos.ID_Cliente = Clientes.ID;
'''
print("Resultado simulado del FULL OUTER JOIN:")
mostrar_tabla(query)


Resultado simulado del FULL OUTER JOIN:


,ID,Nombre,Producto
0,1,Juan,Manzanas
1,1,Juan,Naranjas
2,2,Ana,Plátanos
3,3,Luis,None
4,4,Maria,None
5,5,None,Uvas


## Cierre de la Base de Datos

In [ ]:

# Cerrar conexión
conn.close()
print("Conexión cerrada.")
